# CDP Connection Test

Connect to Chrome DevTools Protocol and verify event capture.

## Prerequisites

1. Edit `jupyter/config.env` and set `CDP_ENABLED=true`
2. Restart JupyterLab: `./start-jupyter.sh`
3. Log into rugs.fun in the CDP browser window

In [ ]:
import sys
from pathlib import Path

# Add lib to path
lib_path = Path.cwd().parent.parent / 'lib'
if str(lib_path) not in sys.path:
    sys.path.insert(0, str(lib_path))

from cdp_notebook import CDPCapture, MockCDPCapture

## 1. Connect to CDP

Connect to Chrome running on port 9222:

In [ ]:
# Use real CDP or mock for testing
USE_MOCK = False  # Set True if Chrome not available

if USE_MOCK:
    capture = MockCDPCapture()
else:
    capture = CDPCapture()

capture.connect()

## 2. Wait for Events

Let it run for 30 seconds to capture some events:

In [ ]:
import time
from IPython.display import clear_output

print("Capturing events for 30 seconds...")
print("Play a game in the CDP browser to generate events.")
print()

for i in range(30):
    clear_output(wait=True)
    print(f"Elapsed: {i+1}s / 30s")
    print(f"Events captured: {len(capture.events)}")
    capture.show_event_distribution()
    time.sleep(1)

print("\nCapture complete!")

## 3. View Captured Events

In [ ]:
# Show recent events
capture.show_recent_events(limit=20)

In [ ]:
# Show event distribution
print("Event Distribution:")
capture.show_event_distribution()

## 4. Inspect Specific Event Type

In [ ]:
# Get gameStateUpdate events
gsu_events = capture.filter_events('gameStateUpdate')
print(f"gameStateUpdate events: {len(gsu_events)}")

if gsu_events:
    print("\nSample event data:")
    import json
    print(json.dumps(gsu_events[-1].get('data', {}), indent=2)[:500])

## 5. Test Recording

In [ ]:
from pathlib import Path
from datetime import datetime

# Start recording
output_dir = Path.home() / 'rugs_recordings' / 'notebook_captures'
output_dir.mkdir(parents=True, exist_ok=True)

session_id = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = output_dir / f'test_capture_{session_id}.jsonl'

capture.start_recording(str(output_file))

In [ ]:
# Wait a bit for more events
print("Recording for 10 seconds...")
time.sleep(10)

# Stop recording
filepath = capture.stop_recording()
print(f"\nRecording saved!")

In [ ]:
# Verify recording
if filepath:
    with open(filepath) as f:
        lines = f.readlines()
    print(f"Recorded {len(lines)} events to {filepath}")

## 6. Cleanup

In [ ]:
capture.disconnect()

## Next Steps

- **02_live_event_monitor.ipynb** - Real-time event monitoring with filters
- **03_batch_capture.ipynb** - Overnight batch capture for knowledge pipeline